In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports

In [49]:
import numpy as np
import jax
import jax.numpy as jnp
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from functools import partial
import os
import sys
import umap
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)
    

In [50]:
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict
from synbio_morpher.utils.misc.string_handling import prettify_keys_for_label
from synbio_morpher.utils.parameter_inference.interpolation_grid import create_parameter_range

In [51]:
sdir = '../data/parameter_based_simulation/2023_08_24_114212'
analytic_name = 'precision'
fn = os.path.join(sdir, [f for f in os.listdir(sdir) if analytic_name in f][0])
config = load_json_as_dict('../data/parameter_based_simulation/2023_08_24_114212/experiment.json')['config_params']

In [52]:
param_range = create_parameter_range(config['parameter_based_simulation'])

x = np.load(fn)

In [53]:
z_data = x[0, :, :, 0, 0, 0, 0]
z_data.shape

(9, 9)

In [54]:



# Read data from a csv

# z_data = pd.DataFrame()

# fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig = go.Figure(data=[go.Surface(z=z_data)])

fig.update_layout(title=prettify_keys_for_label(analytic_name), autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

# fig.show()

In [60]:
n_species = x.shape[0]
sig_idx = 0

slices = [slice(x.shape[1]), slice(x.shape[2]), 0, 0, 0, 0]
xmax = np.max(x[tuple([slice(x.shape[0])] + slices)])

surfs = []
pi = 0
for si in range(n_species):
    if si == sig_idx:
        continue
    surfs.append(go.Surface(z=x[tuple([si] + slices)] + xmax * pi, showscale=False, opacity=0.9))
    pi += 1
    # surfs.append(go.Surface(z=x[si, :, :, 0, 0, 0, 0] + si * xmax, showscale=False, opacity=0.9))

fig = go.Figure(data=surfs)

fig.update_layout(title=prettify_keys_for_label(analytic_name), autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
                  
fig.write_html(f'{analytic_name}.html')

In [79]:

fig = go.Figure()

for i, step in enumerate(param_range):

    slices = [slice(x.shape[1]), slice(x.shape[2]), i, 0, 0, 0]
    xmax = np.max(x[tuple([slice(x.shape[0])] + slices)])
    # surfs = []
    pi = 0
    for si in range(n_species):
        if si == sig_idx:
            continue
        fig.add_trace(
            # surfs.append(
            go.Surface(z=x[tuple([si] + slices)] + xmax * pi,
                       visible=False,
                       name="[3] = " + f'{step:.2e}',
                       showscale=False, opacity=1.0, tickvals=param_range),
        )
        pi += 1
        break
        # surfs.append(go.Surface(z=x[si, :, :, 0, 0, 0, 0] + si * xmax, showscale=False, opacity=0.9))

    # fig.add_trace(surfs)

fig.data[0].visible = True

steps = []
for i, pr in zip(range(len(fig.data)), param_range):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i) + f' - {pr:.2e}'}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Interaction strength: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
    # xaxis=dict(
    #     tickmode='array',
    #     tickvals=param_range
    # ),
    # yaxis=dict(
    #     tickmode='array',
    #     tickvals=param_range
    # )
)

fig.update_xaxes(
    tickmode="array",
    tickvals=np.arange(len(param_range)),
    ticktext=[str(pr) for pr in param_range]
)
fig.write_html('ok.html')

ValueError: Invalid property specified for object of type plotly.graph_objs.Surface: 'tickvals'

Did you mean "cmax"?

    Valid properties:
        autocolorscale
            Determines whether the colorscale is a default palette
            (`autocolorscale: true`) or the palette determined by
            `colorscale`. In case `colorscale` is unspecified or
            `autocolorscale` is true, the default palette will be
            chosen according to whether numbers in the `color`
            array are all positive, all negative or mixed.
        cauto
            Determines whether or not the color domain is computed
            with respect to the input data (here z or surfacecolor)
            or the bounds set in `cmin` and `cmax` Defaults to
            `false` when `cmin` and `cmax` are set by the user.
        cmax
            Sets the upper bound of the color domain. Value should
            have the same units as z or surfacecolor and if set,
            `cmin` must be se